# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/v1/rerank`(cross encoder rerank model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0"
)
## To run qwen2.5-0.5b-instruct model on the Ascend-Npu, you can execute the following command:
# server_process, port = launch_server_cmd(
#     "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0 --device npu --tp 2 --attention-backend torch_native"
# )

wait_for_server(f"http://localhost:{port}")

[2025-07-18 05:06:50] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='0.0.0.0', port=38306, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=956458655, co

[2025-07-18 05:07:09] Attention backend not set. Use fa3 backend by default.
[2025-07-18 05:07:09] Init torch distributed begin.


[2025-07-18 05:07:11] Init torch distributed ends. mem usage=0.00 GB


[2025-07-18 05:07:13] Load weight begin. avail mem=76.29 GB


[2025-07-18 05:07:14] Using model weights format ['*.safetensors']


...

[2025-07-18 05:07:17] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=46.47 GB


[2025-07-18 05:07:17] INFO:     Started server process [250337]
[2025-07-18 05:07:17] INFO:     Waiting for application startup.
[2025-07-18 05:07:17] INFO:     Application startup complete.
[2025-07-18 05:07:17] INFO:     Uvicorn running on http://0.0.0.0:38306 (Press CTRL+C to quit)


[2025-07-18 05:07:17] INFO:     127.0.0.1:54400 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-18 05:07:18] INFO:     127.0.0.1:54412 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-18 05:07:18] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-18T05:07:18.546815


[2025-07-18 05:07:19] INFO:     127.0.0.1:54422 - "POST /generate HTTP/1.1" 200 OK
[2025-07-18 05:07:19] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-07-18 05:07:22] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-18T05:07:22.832184


[2025-07-18 05:07:23] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: False, gen throughput (token/s): 6.57, #queue-req: 0, timestamp: 2025-07-18T05:07:23.114877


[2025-07-18 05:07:23] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, cuda graph: False, gen throughput (token/s): 129.25, #queue-req: 0, timestamp: 2025-07-18T05:07:23.424344


[2025-07-18 05:07:23] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, cuda graph: False, gen throughput (token/s): 129.66, #queue-req: 0, timestamp: 2025-07-18T05:07:23.732831
[2025-07-18 05:07:23] INFO:     127.0.0.1:54434 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.
- `preferred_sampling_params`: The default sampling params specified via `--preferred-sampling-params`. `None` is returned in this example as we did not explicitly configure it in server args.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["preferred_sampling_params"] is None
assert response_json.keys() == {
    "model_path",
    "is_generation",
    "tokenizer_path",
    "preferred_sampling_params",
}

[2025-07-18 05:07:23] INFO:     127.0.0.1:60324 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-07-18 05:07:23] INFO:     127.0.0.1:60330 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-07-18 05:07:23] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-18T05:07:23.873272


[2025-07-18 05:07:24] INFO:     127.0.0.1:60338 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-07-18 05:07:24] INFO:     127.0.0.1:60344 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-07-18 05:07:24] Cache flushed successfully!
[2025-07-18 05:07:24] INFO:     127.0.0.1:60346 - "POST /flush_cache HTTP/1.1" 200 OK


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-07-18 05:07:24] Start update_weights. Load format=auto
[2025-07-18 05:07:24] Update engine weights online from disk begin. avail mem=41.50 GB
[2025-07-18 05:07:24] Using model weights format ['*.safetensors']


[2025-07-18 05:07:25] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  3.52it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  3.51it/s]

[2025-07-18 05:07:25] Update weights end.
[2025-07-18 05:07:25] Cache flushed successfully!
[2025-07-18 05:07:25] INFO:     127.0.0.1:60356 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "qwen/qwen2.5-0.5b-instruct-wrong"
    " (repository not found)."
)

[2025-07-18 05:07:25] Start update_weights. Load format=auto
[2025-07-18 05:07:25] Update engine weights online from disk begin. avail mem=41.46 GB
[2025-07-18 05:07:25] Failed to get weights iterator: qwen/qwen2.5-0.5b-instruct-wrong (repository not found).
[2025-07-18 05:07:25] INFO:     127.0.0.1:60360 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


In [10]:
terminate_process(server_process)

[2025-07-18 05:07:25] Child process unexpectedly failed with exitcode=9. pid=251189
[2025-07-18 05:07:25] Child process unexpectedly failed with exitcode=9. pid=250785


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
embedding_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-1.5B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-18 05:07:35] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-1.5B-instruct', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='0.0.0.0', port=34442, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False,

[2025-07-18 05:07:47] Downcasting torch.float32 to torch.float16.


[2025-07-18 05:07:47] Overlap scheduler is disabled for embedding models.
[2025-07-18 05:07:47] Downcasting torch.float32 to torch.float16.
[2025-07-18 05:07:47] Attention backend not set. Use fa3 backend by default.
[2025-07-18 05:07:47] Init torch distributed begin.


[2025-07-18 05:07:47] Init torch distributed ends. mem usage=0.00 GB


[2025-07-18 05:07:49] Load weight begin. avail mem=62.07 GB


...

Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.28s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.37s/it]

[2025-07-18 05:07:52] Load weight end. type=Qwen2ForCausalLM, dtype=torch.float16, avail mem=58.02 GB, mem usage=4.05 GB.
[2025-07-18 05:07:52] KV Cache is allocated. #tokens: 20480, K size: 0.27 GB, V size: 0.27 GB
[2025-07-18 05:07:52] Memory pool end. avail mem=56.72 GB


[2025-07-18 05:07:52] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=43.62 GB


[2025-07-18 05:07:53] INFO:     Started server process [253397]
[2025-07-18 05:07:53] INFO:     Waiting for application startup.
[2025-07-18 05:07:53] INFO:     Application startup complete.
[2025-07-18 05:07:53] INFO:     Uvicorn running on http://0.0.0.0:34442 (Press CTRL+C to quit)
[2025-07-18 05:07:53] INFO:     127.0.0.1:54614 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-18 05:07:54] INFO:     127.0.0.1:54630 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-18 05:07:54] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-18T05:07:54.463163


[2025-07-18 05:07:54] INFO:     127.0.0.1:54646 - "POST /encode HTTP/1.1" 200 OK
[2025-07-18 05:07:54] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-07-18 05:07:58] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-18T05:07:58.658342
[2025-07-18 05:07:58] INFO:     127.0.0.1:54660 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

[2025-07-18 05:07:58] Child process unexpectedly failed with exitcode=9. pid=254112


## v1/rerank (cross encoder rerank model)
Rerank a list of documents given a query using a cross-encoder model. Note that this API is only available for cross encoder model like [BAAI/bge-reranker-v2-m3](https://huggingface.co/BAAI/bge-reranker-v2-m3) with `attention-backend` `triton` and `torch_native`.


In [14]:
reranker_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path BAAI/bge-reranker-v2-m3 \
    --host 0.0.0.0 --disable-radix-cache --chunked-prefill-size -1 --attention-backend triton --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-18 05:08:09] server_args=ServerArgs(model_path='BAAI/bge-reranker-v2-m3', tokenizer_path='BAAI/bge-reranker-v2-m3', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='BAAI/bge-reranker-v2-m3', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='0.0.0.0', port=32016, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=36718811, constrained_jso

[2025-07-18 05:08:20] Downcasting torch.float32 to torch.float16.


[2025-07-18 05:08:20] Overlap scheduler is disabled for embedding models.
[2025-07-18 05:08:20] Downcasting torch.float32 to torch.float16.
[2025-07-18 05:08:20] Init torch distributed begin.


[2025-07-18 05:08:21] Init torch distributed ends. mem usage=0.00 GB


[2025-07-18 05:08:22] Load weight begin. avail mem=61.47 GB
[2025-07-18 05:08:22] Using model weights format ['*.safetensors']


...

Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.47it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.47it/s]

[2025-07-18 05:08:23] Load weight end. type=XLMRobertaForSequenceClassification, dtype=torch.float16, avail mem=59.86 GB, mem usage=1.61 GB.
[2025-07-18 05:08:23] KV Cache is allocated. #tokens: 20480, K size: 0.94 GB, V size: 0.94 GB
[2025-07-18 05:08:23] Memory pool end. avail mem=57.82 GB


[2025-07-18 05:08:24] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=8194, available_gpu_mem=43.30 GB


[2025-07-18 05:08:24] INFO:     Started server process [256304]
[2025-07-18 05:08:24] INFO:     Waiting for application startup.
[2025-07-18 05:08:24] INFO:     Application startup complete.
[2025-07-18 05:08:24] INFO:     Uvicorn running on http://0.0.0.0:32016 (Press CTRL+C to quit)


[2025-07-18 05:08:25] INFO:     127.0.0.1:38770 - "GET /v1/models HTTP/1.1" 200 OK
[2025-07-18 05:08:25] INFO:     127.0.0.1:38776 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-18 05:08:25] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-18T05:08:25.952155


[2025-07-18 05:08:26] INFO:     127.0.0.1:38792 - "POST /encode HTTP/1.1" 200 OK
[2025-07-18 05:08:26] The server is fired up and ready to roll!


In [15]:
# compute rerank scores for query and documents

url = f"http://localhost:{port}/v1/rerank"
data = {
    "model": "BAAI/bge-reranker-v2-m3",
    "query": "what is panda?",
    "documents": [
        "hi",
        "The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.",
    ],
}

response = requests.post(url, json=data)
response_json = response.json()
for item in response_json:
    print_highlight(f"Score: {item['score']:.2f} - Document: '{item['document']}'")

[2025-07-18 05:08:30] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-18T05:08:30.791547
[2025-07-18 05:08:30] Prefill batch. #new-seq: 1, #new-token: 43, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-18T05:08:30.823004
[2025-07-18 05:08:30] INFO:     127.0.0.1:38802 - "POST /v1/rerank HTTP/1.1" 200 OK


In [16]:
terminate_process(reranker_process)

[2025-07-18 05:08:30] Child process unexpectedly failed with exitcode=9. pid=256811
[2025-07-18 05:08:30] Child process unexpectedly failed with exitcode=9. pid=256610


## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [17]:
# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-18 05:08:41] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='0.0.0.0', port=38874, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, strea

[2025-07-18 05:08:53] Overlap scheduler is disabled for embedding models.
[2025-07-18 05:08:53] Attention backend not set. Use flashinfer backend by default.
[2025-07-18 05:08:53] Init torch distributed begin.


[2025-07-18 05:08:54] Init torch distributed ends. mem usage=0.00 GB


[2025-07-18 05:08:55] Load weight begin. avail mem=77.99 GB


[2025-07-18 05:08:56] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.24it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.08it/s]

[2025-07-18 05:09:00] Load weight end. type=LlamaForSequenceClassification, dtype=torch.bfloat16, avail mem=60.46 GB, mem usage=17.53 GB.


[2025-07-18 05:09:00] KV Cache is allocated. #tokens: 20480, K size: 1.25 GB, V size: 1.25 GB
[2025-07-18 05:09:00] Memory pool end. avail mem=57.66 GB


[2025-07-18 05:09:01] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=57.09 GB


[2025-07-18 05:09:01] INFO:     Started server process [259226]
[2025-07-18 05:09:01] INFO:     Waiting for application startup.
[2025-07-18 05:09:01] INFO:     Application startup complete.
[2025-07-18 05:09:01] INFO:     Uvicorn running on http://0.0.0.0:38874 (Press CTRL+C to quit)
[2025-07-18 05:09:02] INFO:     127.0.0.1:48830 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-18 05:09:02] INFO:     127.0.0.1:48836 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-18 05:09:02] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-18T05:09:02.870419


In [18]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-07-18 05:10:07] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-18T05:10:07.892102
[2025-07-18 05:10:07] INFO:     127.0.0.1:48844 - "POST /encode HTTP/1.1" 200 OK
[2025-07-18 05:10:07] The server is fired up and ready to roll!


[2025-07-18 05:10:22] INFO:     127.0.0.1:37826 - "POST /classify HTTP/1.1" 200 OK


In [19]:
terminate_process(reward_process)

[2025-07-18 05:10:22] Child process unexpectedly failed with exitcode=9. pid=259925
[2025-07-18 05:10:22] Child process unexpectedly failed with exitcode=9. pid=259655


## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [20]:
expert_record_server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0 --expert-distribution-recorder-mode stat"
)

wait_for_server(f"http://localhost:{port}")

[2025-07-18 05:10:32] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen1.5-MoE-A2.7B', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, impl='auto', host='0.0.0.0', port=37178, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=541992449, constrained_js

[2025-07-18 05:10:43] Attention backend not set. Use flashinfer backend by default.
[2025-07-18 05:10:43] Init torch distributed begin.


[2025-07-18 05:10:44] Init torch distributed ends. mem usage=0.00 GB


[2025-07-18 05:10:45] Load weight begin. avail mem=60.24 GB


[2025-07-18 05:10:45] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


...

[2025-07-18 05:10:53] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=28.21 GB


[2025-07-18 05:10:53] INFO:     Started server process [266706]
[2025-07-18 05:10:53] INFO:     Waiting for application startup.
[2025-07-18 05:10:53] INFO:     Application startup complete.
[2025-07-18 05:10:53] INFO:     Uvicorn running on http://0.0.0.0:37178 (Press CTRL+C to quit)


[2025-07-18 05:10:54] INFO:     127.0.0.1:49514 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-18 05:10:54] INFO:     127.0.0.1:49530 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-18 05:10:54] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-18T05:10:54.886818


[2025-07-18 05:10:55] Using default MoE kernel config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-l1b-gpu-23/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/triton_3_3_1/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can create them with https://github.com/sgl-project/sglang/tree/main/benchmark/kernels/fused_moe_triton


In [21]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

[2025-07-18 05:11:10] Resetting ExpertDistributionRecorder...
[2025-07-18 05:11:10] INFO:     127.0.0.1:49556 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-07-18 05:11:10] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-18T05:11:10.374953


[2025-07-18 05:11:10] INFO:     127.0.0.1:49544 - "POST /generate HTTP/1.1" 200 OK
[2025-07-18 05:11:10] The server is fired up and ready to roll!


[2025-07-18 05:11:11] INFO:     127.0.0.1:57884 - "POST /generate HTTP/1.1" 200 OK


[2025-07-18 05:11:11] INFO:     127.0.0.1:57894 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-07-18 05:11:12] Write expert distribution to /tmp/expert_distribution_recorder_1752815472.3841689.pt
[2025-07-18 05:11:12] Resetting ExpertDistributionRecorder...
[2025-07-18 05:11:12] INFO:     127.0.0.1:57898 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [22]:
terminate_process(expert_record_server_process)

[2025-07-18 05:11:12] Child process unexpectedly failed with exitcode=9. pid=267622
[2025-07-18 05:11:12] Child process unexpectedly failed with exitcode=9. pid=267180
